In [1]:
%pylab notebook
import datacube
from datetime import datetime, timedelta
import pandas as pd
import pyproj
import csv

dc = datacube.Datacube(app='nbart-fieldsites')

Populating the interactive namespace from numpy and matplotlib


In [2]:
fieldsites = pd.read_csv(
    'jrsrp_20170430.csv',
    usecols=[
        'obs_key',
        "longitude",
        "latitude",
        "image",
        "band1m",
        "band2m",
        "band3m",
        "band4m",
        "band5m",
        "band6m",
        "band1s",
        "band2s",
        "band3s",
        "band4s",
        "band5s",
        "band6s",
        "count"
    ])

In [3]:
def query_cube(obs_key, lon, lat, image, band1m, band2m, band3m, band4m, band5m, band6m,
        band1s, band2s, band3s, band4s, band5s, band6s, count):
    sensor = 'ls'+image[1:2]
    path = image[8:11]
    row = image[12:15]
    date = image[16:20]+'-'+image[20:22]+'-'+image[22:24]
    target_day = datetime.strptime(date, '%Y-%m-%d')
    day_before = (target_day - timedelta(days=1)).strftime('%Y-%m-%d')
    day_after = (target_day + timedelta(days=1)).strftime('%Y-%m-%d')
    
    query = {'time': (day_before, day_after)}
    
    wgs_84 = pyproj.Proj(init='epsg:4326')
    aus_albers = pyproj.Proj(init='epsg:3577')

    x, y = pyproj.transform(wgs_84, aus_albers, lon, lat)
    
    y_max = y-30
    y_min = y+30
    x_max = x-30
    x_min = x+30
    query['x'] = (x_min, x_max)
    query['y'] = (y_max, y_min)
    query['crs'] = 'EPSG:3577'
        
    #query['source_filter'] = dict(product=sensor+'_nbart_scene', sat_path=path, sat_row=row)
    query['sat_path']=path
    query['sat_row']=row
    query['output_crs'] = 'EPSG:3577'
    query['resolution'] = -25, 25

#     query['measurements'] = ['blue',
#                               'green',
#                               'red', 
#                               'nir',
#                               'swir1', 
#                               'swir2'
#                               ]
    
    if(sensor == 'ls5'):
        query['measurements'] = ['1',
                              '2',
                              '3', 
                              '4',
                              '5', 
                              '7'
                              ]
    if(sensor == 'ls7'):
        query['measurements'] = ['1',
                              '2',
                              '3', 
                              '4',
                              '5', 
                              '7'
                              ]
    
    if(sensor == 'ls8'):
        query['measurements'] = ['2',
                              '3',
                              '4', 
                              '5',
                              '6', 
                              '7'
                              ]
    
    
    data = dc.load(product=sensor+'_nbart_scene', **query)
    
    if(data.coords):
        data = data.squeeze()
        point = data.sel(x=x,y=y, method='nearest')
        new_x = point.coords['x'].values
        new_y = point.coords['y'].values
        
        new_y_max = new_y-30
        new_y_min = new_y+30
        new_x_max = new_x+30
        new_x_min = new_x-30
        query_x = (new_x_min, new_x_max)
        query_y = (new_y_min,new_y_max)
        target_cells = data.sel(x=slice(*query_x),y=slice(*query_y))
        
        x_size = target_cells.coords['x'].to_index().size
        y_size = target_cells.coords['y'].to_index().size
        
        if(sensor == 'ls5'):
            target_cells['blue'] = getattr(target_cells, '1')
            target_cells['green'] = getattr(target_cells, '2')
            target_cells['red'] = getattr(target_cells, '3')
            target_cells['nir'] = getattr(target_cells, '4')
            target_cells['swir1'] = getattr(target_cells, '5')
            target_cells['swir2'] = getattr(target_cells, '7')
        if(sensor == 'ls7'):
            target_cells['blue'] = getattr(target_cells, '1')
            target_cells['green'] = getattr(target_cells, '2')
            target_cells['red'] = getattr(target_cells, '3')
            target_cells['nir'] = getattr(target_cells, '4')
            target_cells['swir1'] = getattr(target_cells, '5')
            target_cells['swir2'] = getattr(target_cells, '7')

        if(sensor == 'ls8'):
            target_cells['blue'] = getattr(target_cells, '2')
            target_cells['green'] = getattr(target_cells, '3')
            target_cells['red'] = getattr(target_cells, '4')
            target_cells['nir'] = getattr(target_cells, '5')
            target_cells['swir1'] = getattr(target_cells, '6')
            target_cells['swir2'] = getattr(target_cells, '7')
        
        blue_mean = float(target_cells.blue.mean())
        blue_std = float(target_cells.blue.std())
        green_mean = float(target_cells.green.mean())
        green_std = float(target_cells.green.std())
        red_mean = float(target_cells.red.mean())
        red_std = float(target_cells.red.std())
        nir_mean = float(target_cells.nir.mean())
        nir_std = float(target_cells.nir.std())
        swir1_mean = float(target_cells.swir1.mean())
        swir1_std = float(target_cells.swir1.std())
        swir2_mean = float(target_cells.swir2.mean())
        swir2_std = float(target_cells.swir2.std())
        
        min_lon, min_lat = pyproj.transform(aus_albers, wgs_84, new_x_min, new_y_min)
        max_lon, max_lat = pyproj.transform(aus_albers, wgs_84, new_x_max, new_y_max)
        
        if(-999 in [blue_mean,green_mean,red_mean,nir_mean,swir1_mean,swir2_mean] or 0 in [blue_std,green_std,red_std,nir_std,swir1_std,swir2_std]):
            return 'n/a', 'n/a', 'n/a', 'n/a', band1m, band2m, band3m, band4m, band5m, band6m, band1s, band2s, band3s, band4s, band5s, band6s, count
        
        
        return min_lon, min_lat, max_lon, max_lat, blue_mean, green_mean, red_mean, nir_mean, swir1_mean, swir2_mean, blue_std, green_std, red_std, nir_std, swir1_std, swir2_std, x_size * y_size

    else:
        return 'n/a', 'n/a', 'n/a', 'n/a', band1m, band2m, band3m, band4m, band5m, band6m, band1s, band2s, band3s, band4s, band5s, band6s, count

In [4]:
with open('nbart_fieldsites_with_jrsrp_substitutes_discarding_bad.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    columns = ['obs_key',
        'longitude',
        'latitude',
        'image',
        'min_lon',
        'min_lat',
        'max_lon',
        'max_lat',
        'blue_mean', 
        'green_mean',
        'red_mean',
        'nir_mean',
        'swir1_mean',
        'swir2_mean',
        'blue_std',
        'green_std',
        'red_std',
        'nir_std',
        'swir1_std',
        'swir2_std',
        'count'
        ]
    writer.writerow(columns)
    fieldsite_tuples = [tuple(x) for x in fieldsites.values]
    for fieldsite in fieldsite_tuples: 
        partialrow=(fieldsite[0],fieldsite[1],fieldsite[2],fieldsite[3])
        writer.writerow(partialrow+ query_cube(*fieldsite))